# 0. Library Import and Set the Directory

In [47]:
import dataset
import models
import csv
import os
import torch
import torchvision

from PIL import Image

import pandas as pd

In [14]:
test_dir = '/opt/ml/input/data/eval'

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"torch version: {torch.__version__}")
print(f"device: {device}")

torch version: 1.6.0
device: cuda:0


# 1. Model Setting

In [10]:
def load_ckp(checkpoint_fpath, model, optimizer, device):
    checkpoint = torch.load(checkpoint_fpath, map_location = device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

In [34]:
A_model_path = f"./ckpt/{A_model_ckpt}"
B_model_path = f"./ckpt/{B_model_ckpt}"
C_model_path = f"./ckpt/{C_model_ckpt}"

In [35]:
A_model = models.EffiB3Based(num_classes=2)
B_model = models.EffiB3Based(num_classes=3)
C_model = models.EffiB3Based(num_classes=2)

# 2. Dataset Load

In [30]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))

In [31]:
image_dir = os.path.join(test_dir, 'images')

In [32]:
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

transformations = dataset.get_transforms(('val'))

pred_data = dataset.MaskDataset(image_paths, (lambda x:-1),transformations['val'])

loader = torch.utils.data.DataLoader(
    pred_data,
    shuffle=False
)


# 3. Inference

In [33]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
B_model, _, _ = load_ckp(B_model_path, B_model, torch.optim.Adam(B_model.parameters(), lr=1), device)
B_model.eval()
B_model.to(device)

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
B_predictions = []
for images, label in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = B_model(images)
        pred = pred.argmax(dim=-1)
        B_predictions.extend(pred.cpu().numpy())

C_model, _, _ = load_ckp(C_model_path, C_model, torch.optim.Adam(C_model.parameters(), lr=1), device)
C_model.eval()
C_model.to(device)

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
C_predictions = []
for images, label in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = C_model(images)
        pred = pred.argmax(dim=-1)
        C_predictions.extend(pred.cpu().numpy())


A_detail_model, _, _ = load_ckp(A_model_path, A_model, torch.optim.SGD(A_model.parameters(), lr=1e-3), device)
A_detail_model.eval()
A_detail_model.to(device)

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
A_predictions = []
for images, label in pred_iter_for_age:
    with torch.no_grad():
        images = images.to(device)
        pred = A_model(images)
        pred = pred.argmax(dim=-1)
        A_predictions.extend(pred.cpu().numpy())

        
all_predictions = []

for pred_tuple in zip(B_predictions, C_predictions, A_predictions):
    predicted_ans = pred_tuple[0]*6 + pred_tuple[1]*3 + (0 if pred_tuple[2]<3 else (1 if pred_tuple[2]<9 else 2))
    all_predictions.append(predicted_ans)
    
        
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_3.csv'), index=False)
print('test inference is done!')

test inference is done!
